In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.cm import get_cmap
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
#from datatools import wfip2
#from datatools.remote_sensing import ESRL_wind_profiler, Vaisala_CL31
from mmctools.dataloaders import *
from mmctools.measurements.radar import profiler
from mmctools.measurements.lidar import Vaisala_CL31

ImportError: cannot import name 'profiler' from 'mmctools.measurements.radar' (/Users/equon/a2e-mmc/mmctools/measurements/radar.py)

# Compare PBL height (mixing height) observations
also see previous analysis: `~/WFIP2/PBL_characterization_2016-11-21`

## read radar.z04.b0 data
Radar • ESRL Wind Profiler with RASS, Wasco Airport • Reviewed Data

45.59011, -120.67193

Averaged at interval of 60 minutes at the *start* of the bin

In [ ]:
elev = 456. # [m]

In [ ]:
wind = [
    pd.read_csv('data/Wasco_radar_wind_mode{:d}.csv'.format(i),parse_dates=['date_time']).set_index('date_time')
    for i in range(2)
]
rass = pd.read_csv('data/Wasco_radar_rass.csv',parse_dates=['date_time']).set_index('date_time')

In [ ]:
wind[0].head()

In [ ]:
wind[1].head()

In [ ]:
rass.head()

In [ ]:
z = rass['HT'].unique()

## calculate virtual potential temperature
NOAA approach (from Irina Djalalova, 2019-01-31):
- corrected temperature is _not_ used (correction is only for vertical velocity and has no appreciable effect; radio frequency interference and outliers are not accounted for)
- assumes standard atmosphere (http://glossary.ametsoc.org/wiki/Standard_atmosphere)

Equations:

1. Barometric formula for an exponential atmosphere with nonzero lapse rate
$p = p_0 \left[\frac{T_0 - \Gamma(h-h_0)}{T_0}\right]^{1/(\Gamma R/g)}
= p_0 \left[\frac{T_0/\Gamma - h_{asl}}{T_0/\Gamma}\right]^{1/(\Gamma R/g)}$
2. Poisson equation - the relationship between temperature T and pressure p of an ideal gas undergoing an adiabatic process

Notes:
- lapse rate ($\Gamma$) is 6.5$^\circ$C/km, $\Gamma R/g$ = 0.19
- temperature at zero pressure altitude is 15 deg C; $T_0/\Gamma$ = 44331 m
- for one standard atmosphere, $p_0$ = 101,325 Pa; Eqn 1 can also be written
$p = 100\left(\frac{44331-h_{asl}}{11880}\right)^{1/0.19}
= 100\left(1013.25^{0.19}\frac{44331-h_{asl}}{44331}\right)^{1/0.19}
= p_0\left(\frac{44331-h_{asl}}{44331}\right)^{1/0.19}$
- $R/c_p \approx R_d/C_{pd} \approx 287./1004. = 0.286$

In [ ]:
rass['P'] = 100.*((44331.514 - (rass['HT']*1000.+elev))/11880.516)**(1./0.1902632)

In [ ]:
Pref = 100000. # reference pressure [Pa]
rass['VPT'] = ((rass['T']+273.16)*(Pref/rass['P'])**0.286) # [K]

In [ ]:
rass_clean = rass.loc[rass['QC_T']==0]
len(rass),len(rass_clean)

## check Nov 21, 2016

In [ ]:
tstart,tend = pd.to_datetime('2016-11-21 17:00'), pd.to_datetime('2016-11-22 05:00')
#datetime_range = (rass.index >= tstart) & (rass.index <= tend)

### first, check potential temperature profile

In [ ]:
datetime_range = (rass.index >= tstart-pd.to_timedelta(10,unit='m')) & (rass.index <= tend)
rass_nov21 = rass.loc[datetime_range].pivot(columns='HT',values='VPT')

In [ ]:
rass_nov21.head()

In [ ]:
len(rass_nov21),rass_nov21.index

In [ ]:
tt,zz = np.meshgrid(rass_nov21.index.unique(),z,indexing='ij')

In [ ]:
fig,ax = plt.subplots(figsize=(11,4))
pcm = ax.pcolormesh(tt,1000*zz,rass_nov21)
fig.colorbar(pcm)
ax.set_xlabel('UTC')
ax.set_ylabel('height AGL [m]')

ax.set_xticks(pd.date_range(tstart,tend,freq='H'))#, minor=True)
ax.xaxis.set_major_locator(mdates.HourLocator())
#ax.xaxis.set_major_locator(mdates.DayLocator())
#ax.xaxis.set_minor_locator(mdates.HourLocator())
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-$b-%y'))
#ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H'))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H'))
ax.set_title('{:s} to {:s}'.format(str(tstart.date()),str(tend.date())))

In [ ]:
fig,ax = plt.subplots(figsize=(4,8))
cm = get_cmap('viridis')
Tperiod = (rass_nov21.index[-1] - rass_nov21.index[0]).total_seconds()
for ti,row in rass_nov21.iterrows():
    tfrac = (ti - rass_nov21.index[0]).total_seconds() / Tperiod
    if (tfrac==0) or (tfrac==1):
        label = str(ti)
    else:
        label = ''
    ax.plot(row.values,1000*row.index,color=cm(tfrac),label=label)
ax.set_xlabel(r'$\theta_v$')
ax.set_ylabel('height AGL [m]')
ax.legend()

### check signal-to-noise

In [ ]:
datetime_range = [
    (wind[i].index >= tstart) & (wind[i].index <= tend)
    for i in range(2)
]
snr0_nov21 = [
    wind[i].loc[datetime_range[i]].pivot(columns='HT',values='SNR.0')
    for i in range(2)
]
snr1_nov21 = [
    wind[i].loc[datetime_range[i]].pivot(columns='HT',values='SNR.1')
    for i in range(2)
]
snr2_nov21 = [
    wind[i].loc[datetime_range[i]].pivot(columns='HT',values='SNR.2')
    for i in range(2)
]

In [ ]:
for i in range(2):
    print(len(snr0_nov21[i]),snr0_nov21[i].index)

In [ ]:
for i in range(2):
    fig,ax = plt.subplots(figsize=(6,6))
    ax.plot(snr0_nov21[i].values.ravel(),snr1_nov21[i].values.ravel(),'.',label='SNR0,1')
    ax.plot(snr0_nov21[i].values.ravel(),snr2_nov21[i].values.ravel(),'.',label='SNR0,2')
    ax.plot(snr1_nov21[i].values.ravel(),snr2_nov21[i].values.ravel(),'.',label='SNR1,2')
    ax.plot([-25,5],[-25,5],'k--')
    ax.legend()
    ax.set_title('mode {:d}'.format(i))

In [ ]:
fig,ax = plt.subplots(ncols=2,sharey=True,figsize=(8,8))
cm = get_cmap('viridis')
for i in range(2):
    Tperiod = (snr0_nov21[i].index[-1] - snr0_nov21[i].index[0]).total_seconds()
    for ti,row in snr0_nov21[i].iterrows():
        tfrac = (ti - snr0_nov21[i].index[0]).total_seconds() / Tperiod
        #if (tfrac==0) or (tfrac==1):
        #    label = str(ti)
        #else:
        #    label = ''
        label = str(ti.strftime('%H:%M'))
        ax[i].plot(row.values,1000*row.index,color=cm(tfrac),label=label)
        ax[i].set_title('mode {:d}'.format(i))
        ax[i].legend()
ax[0].set_xlabel(r'signal-to-noise ratio')
ax[1].set_xlabel(r'signal-to-noise ratio')
ax[0].set_ylabel('height AGL [m]')
fig.suptitle('beam 0')

In [ ]:
# average 3 beams
snr_nov21 = [
    (snr0_nov21[i] + snr1_nov21[i] + snr2_nov21[i])/3
    for i in range(2)
]

In [ ]:
fig,ax = plt.subplots(ncols=2,sharey=True,figsize=(8,8))
cm = get_cmap('viridis')
for i in range(2):
    Tperiod = (snr_nov21[i].index[-1] - snr_nov21[i].index[0]).total_seconds()
    for ti,row in snr_nov21[i].iterrows():
        tfrac = (ti - snr_nov21[i].index[0]).total_seconds() / Tperiod
        #if (tfrac==0) or (tfrac==1):
        #    label = str(ti)
        #else:
        #    label = ''
        label = str(ti.strftime('%H:%M'))
        ax[i].plot(row.values,1000*row.index,color=cm(tfrac),label=label)
        ax[i].set_title('mode {:d}'.format(i))
        ax[i].legend()
ax[0].set_ylabel('height AGL [m]')
ax[0].set_xlabel(r'signal-to-noise ratio')
ax[1].set_xlabel(r'signal-to-noise ratio')
fig.suptitle('average of 3 beams')

### read ceilometer data for comparison

In [ ]:
ceilo = pd.read_csv('data/Wasco_ceilometer.csv',parse_dates=['date_time']).set_index('date_time')

In [ ]:
# throw out unused cloud info
ceilo = ceilo.drop(columns=['Status','Height1','Height2','Height3'])

In [ ]:
# reshape data 
ceilo = ceilo.stack().reset_index(1).rename(columns={'level_1':'height',0:'backscat'})

In [ ]:
# trim values above say, 2 km or so
ceilo['height'] = ceilo['height'].astype(float)
ceilo = ceilo.loc[ceilo['height'] <= 2500.]
ceilo_heights = ceilo['height'].unique()

In [ ]:
# do some averaging
ceilo = ceilo.pivot(columns='height',values='backscat').resample('30min',closed='left').mean()

In [ ]:
ceilo

### 1-hr near-neutral period (SciTech 2019 study)

In [ ]:
signals = ['SNR.0','SNR.1','SNR.2']
inrange = (wind[0].index >= '2016-11-21 22:00') & (wind[0].index <= '2016-11-21 23:15')
heights = 1000 * wind[0].loc[inrange,'HT'].unique()
inrange1 = (wind[1].index >= '2016-11-21 22:00') & (wind[1].index <= '2016-11-21 23:15')
heights1 = 1000 * wind[1].loc[inrange1,'HT'].unique()

In [ ]:
fig,ax = plt.subplots(figsize=(4,8))
df0 = wind[0].loc[inrange,signals]
for t in df0.index.unique():
    for sig in signals:
        ax.plot(df0.loc[df0.index==t,sig],heights)
ax.axhline(600,color='k',ls='--')
ax.set_xlabel(r'signal-to-noise ratio',fontsize='large')
ax.set_ylabel('height AGL [m]',fontsize='large')
ax.set_title('Radar Wind Profiler',fontsize='x-large')

In [ ]:
ceilo_inrange = (ceilo.index >= '2016-11-21 22:00') & (ceilo.index <= '2016-11-21 23:00')

In [ ]:
fig,ax = plt.subplots(figsize=(8,8),ncols=2,sharey=True)
df0 = wind[0].loc[inrange,signals]
colorlist = [colors[i] for i in (0,2)]
for itime,t in enumerate(df0.index.unique()):
    df = df0.loc[df0.index==t]
    color = colorlist[itime]
    ax[0].plot(df.mean(axis=1),heights,color=color,label=str(t))
    ax[0].fill_betweenx(heights,df.min(axis=1),df.max(axis=1),color=color,alpha=0.2)
ax[0].axhline(600,color='k',lw=1,alpha=0.5)
ax[0].set_ylim((0,2000))
ax[0].legend()
ax[0].set_ylabel('height AGL [m]',fontsize='large')
ax[0].set_xlabel('signal-to-noise ratio [-]',fontsize='large')
ax[0].set_title('Radar Wind Profiler',fontsize='x-large')

df = ceilo.loc[ceilo_inrange]
for itime,(t,rowdata) in enumerate(df.iterrows()):
    ax[1].plot(rowdata,ceilo_heights,color=colors[itime],label=str(t))
ax[1].axhline(600,color='k',lw=1,alpha=0.5)
ax[1].set_xlim((-15,30))
ax[1].legend()
ax[1].set_xlabel(r'backscatter [10$^{-4}$srad$^{-1}$km$^{-1}$]',fontsize='large')
ax[1].set_title('Ceilometer',fontsize='x-large')

In [ ]:
# backscatter profile filtered to radar resolution (~60 m ==> 6 pts w/ 10-m spacing)
N = 6
fltr = np.ones((N,)) / N

fig,ax = plt.subplots(figsize=(12,8),ncols=3,sharey=True)

spd0 = wind[0].loc[inrange,['SPD']]
spd1 = wind[1].loc[inrange1,['SPD']]
snr = wind[0].loc[inrange,signals]
colorlist = [colors[i] for i in (0,2)]
for itime,t in enumerate(spd0.index.unique()):
    df = spd0.loc[t]
    ax[0].plot(df.mean(axis=1),heights,color=colorlist[itime],label=str(t))
for itime,t in enumerate(spd1.index.unique()):
    df = spd1.loc[t]
    ax[0].plot(df.mean(axis=1),heights1,color=colorlist[itime],ls='--')
for itime,t in enumerate(snr.index.unique()):
    df = snr.loc[t]
    color = colorlist[itime]
    ax[1].plot(df.mean(axis=1),heights,color=color,label=str(t))
    ax[1].fill_betweenx(heights,df.min(axis=1),df.max(axis=1),color=color,alpha=0.2)
ax[0].axhline(600,color='k',lw=1,alpha=0.5)
ax[1].axhline(600,color='k',lw=1,alpha=0.5)
ax[0].set_ylim((0,2000))
ax[0].set_ylabel('height AGL [m]',fontsize='large')
ax[0].legend()
ax[1].legend()
ax[0].set_xlabel('wind speed [m/s]',fontsize='large')
ax[0].set_title('Radar Wind Profiler',fontsize='x-large')
ax[1].set_xlabel('signal-to-noise ratio [-]',fontsize='large')
ax[1].set_title('Radar Wind Profiler',fontsize='x-large')

df = ceilo.loc[ceilo_inrange]
zvals = ceilo_heights[int(N/2):-int(N/2)+1]
for itime,(t,rowdata) in enumerate(df.iterrows()):
    #ax[1].plot(rowdata,ceilo_heights,color=colors[itime],label=str(t))
    bs_filtered = np.convolve(rowdata,fltr,mode='valid)')
    ax[2].plot(bs_filtered,zvals,color=colors[itime],label=str(t))
ax[2].axhline(600,color='k',lw=1,alpha=0.5)
ax[2].set_xlim((-15,30))
ax[2].legend()
ax[2].set_xlabel(r'backscatter [10$^{-4}$srad$^{-1}$km$^{-1}$]',fontsize='large')
ax[2].set_title('Ceilometer (filtered)',fontsize='x-large')
fig.savefig('figures/Wasco_PBL_height_20161121_22Z.png',dpi=150,bbox_inches='tight')